original idea

In [ ]:
import random
from midiutil import MIDIFile

track = 0
channel = 0
tempo = 120
time = 0

MyMIDI = MIDIFile(1)
MyMIDI.addTempo(track, time, tempo)
MyMIDI.addProgramChange(track, channel, time, 0)

class Note:
    def __init__(self, pitch, velocity, duration):
        self.pitch = pitch
        self.velocity = velocity
        self.duration = duration
        
    def __str__(self):
        return "p: %s, v: %s, d: %s" % (self.pitch, self.velocity, self.duration)
    
    def __repr__(self):
        return str(self)

class SlidingWindow:
    def __init__(self, input_list):
        self.list_ = input_list
        self.start    = 0
        self.length  = 2
        
    def update_start(self):
        if random.random() > 0.3: 
            if self.start < (len(self.list_) - 1):
                self.start = self.start + 1
        else:
            if self.start > 0:
                self.start = self.start - 1       
        
    def update_length(self):    
        # calculate length
        if random.random() > 0.3:
            if self.start + self.length + 1 < (len(self.list_) - 1):
                self.length = self.length + 1
        else:
            if self.length > 4:
                self.length = self.length - 1            
        
    def move(self):
        # calculate start move
        if random.random() > 0.5:
            self.update_start()
        else:
            self.update_length()
            
        #print("start", self.start, "length", self.length)
            
    def get_windowed_list(self):
        return self.list_[self.start : self.start + self.length]
        
    
# pitch_list = list(range(1, 37))
pitch_list = [0, 8, 0, 7, 14, 3, 2, 0, 12, 12, 0, 19, 7, 6, 7, 5, 7]
note_list = []
window = SlidingWindow(pitch_list)

for i in range(80):
    num_repetitions = random.randint(3, 6)
    seq = window.get_windowed_list()
    print(seq)
    
    for rep in range(num_repetitions):
        velocity = 100 # accent first note of each
        for p in seq:
            note_list.append(Note(p, velocity, duration))
            velocity = 50
            
    window.move()
    
time = 0
for note in note_list:
    MyMIDI.addNote(track, channel, note.pitch + 72, time, note.duration, note.velocity)
    time = time + note.duration
    
### do it again

note_list = []
window = SlidingWindow(pitch_list)

for i in range(80):
    num_repetitions = random.randint(3, 6)
    seq = window.get_windowed_list()
    print(seq)
    
    for rep in range(num_repetitions):
        velocity = 100
        for p in seq:
            note_list.append(Note(p, velocity, duration))
            velocity = 50
        
    window.move()
    
time = 0
for note in note_list:
    MyMIDI.addNote(track, channel, note.pitch + 48, time, note.duration, note.velocity)
    time = time + note.duration
    
    
    
with open("sliding_window.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

rhythmic accent (longer duration) on first note of each phrase

In [84]:
import random
from midiutil import MIDIFile

track = 0
channel = 0
tempo = 120
time = 0

MyMIDI = MIDIFile(1)
MyMIDI.addTempo(track, time, tempo)
MyMIDI.addProgramChange(track, channel, time, 0)

class Note:
    def __init__(self, pitch = 60, velocity = 60, duration = 1, rest = False):
        self.pitch = pitch
        self.velocity = velocity
        self.duration = duration
        self.start_time_sec = 0
        self.rest = rest
        
    def __str__(self):
        return "p: %s, v: %s, d: %s r?: %s" % (self.pitch, self.velocity, self.duration, self.rest)
    
    def __repr__(self):
        return str(self)

class NoteList:
    def __init__(self):
        self.note_list = []
    
    def addNote(self, note):
        self.note_list.append(note)
        if len(self.note_list) > 0:
            self.note_list[-1].start_time_sec = self.note_list[-2].start_time_sec + self.note_list[-2].duration
                
    def getNotesWithRestsRemoved(self):
        return [n for n in self.note_list if n.rest == False]
    
class SlidingWindow:
    def __init__(self, input_list):
        self.list_ = input_list
        self.start    = 0
        self.length  = 2
        
    def update_start(self):
        if random.random() > 0.3: 
            if self.start < (len(self.list_) - 1):
                self.start = self.start + 1
        else:
            if self.start > 0:
                self.start = self.start - 1       
        
    def update_length(self):    
        # calculate length
        if random.random() > 0.3:
            if self.start + self.length + 1 < (len(self.list_) - 1):
                self.length = self.length + 1
        else:
            if self.length > 4:
                self.length = self.length - 1            
        
    def move(self):
        # calculate start move
        if random.random() > 0.5:
            self.update_start()
        else:
            self.update_length()
            
        # print("start", self.start, "length", self.length)
            
    def get_windowed_list(self):
        return self.list_[self.start : self.start + self.length]
        
pitch_list = [0, 8, 0, 7, 14, 3, 2, 0, 12, 12, 0, 19, 7, 6, 7, 5, 7]

note_list_r = []
window_r = SlidingWindow(pitch_list)

note_list_l = []
window_l = SlidingWindow(pitch_list)

num_phrases = 80

for phrase in range(num_phrases):
    ### right hand
    num_repetitions = random.randint(3, 6)
    seq = window_r.get_windowed_list()
    # print(seq)
    
    section = 0
    if phrase < 10:
        section = 1
    elif phrase < 20:
        section = 2
    elif phrase < 30:
        section = 3
    elif phrase < 40:
        section = 4
        
    rest_first = False  
    stutter = False
    init_duration = 0.25
        
    if section == 1:
        stutter = True
    elif section == 2:
        if random.random() > 0.33:
            stutter = True
    elif section == 3:
        init_duration = 1
    elif section == 4:
        rest_first = True
        
    duration = init_duration
    for rep in range(num_repetitions):
        velocity = 100 # accent first note of each
        if rest_first:
            note_list_r.append(Note(rest = True, duration = duration))
        for p in seq:
            if stutter and random.random() > 0.5:
                note_list_r.append(Note(p, velocity, duration, rest = True))
            else:
                note_list_r.append(Note(p, velocity, duration))
            velocity = 50
            duration = 0.25
        
    window_r.move()
    
    ## left hand
    num_repetitions = random.randint(3, 6)
    seq = window_l.get_windowed_list()
    # print(seq)
    
    duration = init_duration
    for rep in range(num_repetitions):
        velocity = 100
        if rest_first:
            note_list_l.append(Note(rest = True, duration = duration))
        for p in seq:
            if stutter and random.random() > 0.5:
                note_list_l.append(Note(p, velocity, duration, rest = True))
            else:
                note_list_l.append(Note(p, velocity, duration))
            velocity = 50
            duration = 0.25
            
    window_l.move()
    
time = 0
for note in note_list_l:
    if not note.rest:
        MyMIDI.addNote(track, channel, note.pitch + 72, time, note.duration, note.velocity)
    time = time + note.duration
    
time = 0
for note in note_list_r:
    if not note.rest:
        MyMIDI.addNote(track, channel, note.pitch + 48, time, note.duration, note.velocity)
    time = time + note.duration
    
with open("sliding_window.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

[p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?: False, p: 8, v: 50, d: 0.25 r?: False, p: 0, v: 50, d: 0.25 r?: False, p: 0, v: 100, d: 0.25 r?:

In [60]:
a = Note(rest = True)

In [61]:
print(a)

p: 60, v: 60, d: 1 r?: False
